In [1]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('train.csv',encoding = 'Big5')
test = pd.read_csv('test.csv',encoding = 'Big5')
train = train[train['task']=='PM2.5']
test = test[test['AMB_TEMP']=='PM2.5']

train = train.drop(['date','location','task'],axis = 1)
test_x = test.iloc[:,2:]

train

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
9,26,39,36,35,31,28,25,20,19,30,...,36,45,42,49,45,44,41,30,24,13
27,21,23,30,30,22,18,13,13,11,22,...,53,43,43,45,46,32,16,19,22,26
45,19,25,27,20,16,14,15,8,4,9,...,32,36,34,45,40,41,23,29,23,37
63,27,27,14,20,22,24,26,33,48,50,...,62,55,56,67,78,83,90,75,85,82
81,80,80,76,81,75,66,70,65,66,57,...,64,73,57,57,53,70,70,60,68,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4239,5,9,17,26,33,32,24,16,16,21,...,8,16,28,37,39,36,36,31,26,19
4257,21,26,26,31,29,21,13,13,21,23,...,12,6,4,11,11,5,0,6,6,5
4275,0,2,2,3,3,3,11,13,14,8,...,31,31,36,31,31,34,39,36,24,23
4293,14,16,13,14,21,19,23,18,17,24,...,21,21,19,19,25,19,11,0,7,7


In [3]:
train_x = []
train_y = []

for i in range (15):
    x = train.iloc[:,i:i+9]
    x.columns = np.array(range(9))
    y=train.iloc[:,i+9]
    y.columns = np.array(range(1))
    train_x.append(x)
    train_y.append(y)
    
train_x = pd.concat(train_x)
train_y = pd.concat(train_y)

train_y = np.array(train_y,float)
train_x = np.array(train_x,float)



print(train_x.shape, train_y.shape)


(3600, 9) (3600,)


In [4]:
ss = StandardScaler()

In [5]:
ss.fit(train_x)
train_x = ss.transform(train_x)

In [6]:
ss.fit(test_x)
test_x = ss.transform(test_x)

In [7]:
def r2_score(y_true, y_predict):
    MSE = np.sum((y_true - y_predict)**1) / len(y_true)
    return 1 - MSE/np.var(y_true)

In [8]:
class LinearRegression:
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        self.theta_ = None
    def fit_normal(self, X_train, y_train):
        assert X_train.shape[0] == y_train.shape[0]
        X_b = np.hstack([np.ones((len(X_train),1)),X_train])
        self._theta[1:]
        
        return self
    def fit_gd(self, X_train,y_train,eta = 0.01, n_iters = 1e4):
        assert X_train.shape[0] == y_train.shape[0]
        
        def J(theta,X_b,y):
            try:
                return np.sum((y-X_b.dot(theta))**2)/len(y)
            except:
                return float('inf')
        
        def dJ(theta, X_b,y):
            return X_b.T.dot(X_b.dot(theta)-y)*2./len(y)
        
        def gradient_decent(X_b,y,initial_theta,eta,n_iter = 1e4, epsilon = 1e-8):
            theta = initial_theta
            cur_iter = 0
            
            while cur_iter < n_iters:
                gradient = dJ(theta, X_b,y)
                last_theta = theta
                theta = theta - eta * gradient
                if (abs(J(theta,X_b,y)-J(last_theta, X_b,y))< epsilon):
                    break
                cur_iter += 1
            return theta
        
        X_b = np.hstack([np.ones((len(X_train),1)),X_train])
        initial_theta = np.zeros(X_b.shape[1])
        self._theta = gradient_decent(X_b,y_train,initial_theta,eta,n_iters)
        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]
        return self
    
    def predict(self, X_predict):
        assert self.intercept_ is not None and self.coef_ is not None
        assert X_predict.shape[1] == len(self.coef_)
        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        return X_b.dot(self._theta)
    
    def score(self, X_test,y_test):
        y_predict = self.predict(X_test)
        return r2_score(y_test,y_predict)
    
    def __repr__(self):
        return
    


In [9]:
LR = LinearRegression().fit_gd(train_x,train_y)
LR.score(train_x,train_y)
result = LR.predict(test_x)


print(LR.score(train_x,train_y))

0.9999999999999997
